# The dojo

## Setup training device

In [1]:
!nvidia-smi

Fri Mar 10 15:59:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 25%   57C    P0    72W / 250W |    579MiB / 11170MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
from torch import device, nn
import torchvision


print(f'PyTorch version: {torch.__version__}\ntorchvision version: {torchvision.__version__}')

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Using device: {device}')

PyTorch version: 1.13.1+cu117
torchvision version: 0.13.1
Using device: cuda


/home/novem-desktop/miniconda3/envs/cv/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/novem-desktop/miniconda3/envs/cv/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


## Data loading

In [3]:
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader

MNIST_train_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    target_transform=None,
)

MNIST_test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.img_names = pd.read_csv(img_dir + "/../../" + annotations_file)        
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):        
        img_path = os.path.join(self.img_dir, self.img_names.iloc[idx, 0])
        image = read_image(img_path)        
        if self.transform:
            image = self.transform(image)
        return image.float()


MNIST_class_names = MNIST_train_data.classes

train_data = CustomImageDataset("indices/trainIndex/bremen.csv","data/leftImg8bit/train/bremen/")
test_data = CustomImageDataset("indices/testIndex/berlin.csv","data/leftImg8bit/test/berlin/")

# Turn data into batches (mini-batches)
# More computationally efficent, so we dont store all data in memory
# Gives neural network more changes to update its gradients per epoch
# andrew ng minibatches

BATCH_SIZE = 32

train_dataloader = DataLoader(train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

test_dataloader = DataLoader(test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

MNIST_train_dataloader = DataLoader(MNIST_train_data,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              drop_last=True) # Drop last

MNIST_test_dataloader = DataLoader(MNIST_test_data,
                             batch_size=BATCH_SIZE,
                             shuffle=False)

print(f'Len of train dataloader: {len(MNIST_train_dataloader)} batches of {BATCH_SIZE}')
print(f'Len of test dataloader: {len(MNIST_test_dataloader)} batches of {BATCH_SIZE}')

Len of train dataloader: 12000 batches of 5
Len of test dataloader: 2000 batches of 5


### Check out what's inside the training dataloader

In [4]:
train_features_batch = next(iter(MNIST_train_dataloader))
MNIST_train_features_batch, MNIST_train_labels_batch = next(iter(MNIST_train_dataloader))

## Model

### Create a flatten layer - Testing Flatten()

In [5]:
flatten_model = nn.Flatten()

x = train_features_batch[0]
print(f'Shape before flattening: {x.shape}')     # torch.Size([3, 1024, 2048])

output = flatten_model(x)
print(f'Shape after flattening: {output.shape}') # torch.Size([3, 2097152])

Shape before flattening: torch.Size([5, 1, 28, 28])
Shape after flattening: torch.Size([5, 784])


### Instantiate model

#### Baseline models

In [6]:
# For testing - do not change
from model.chrome_vision import ChromeVisionModel

test_CPU_baseline_model = ChromeVisionModel(
    input_shape=784,    
    hidden_units=10,    # Units in the hidden layer
    output_shape=len(MNIST_class_names) # one for every class (Arbitrary right now)
).to(device)

model = test_CPU_baseline_model

In [7]:
# For testing - do not change
from model.chrome_vision import ChromeVisionModelV2

test_GPU_CNN_model = ChromeVisionModelV2(
    input_shape=1,      # Color channel: black/white 
    hidden_units=10,    # Units in the hidden layer
    output_shape=(len(MNIST_class_names)) # Each class
).to(device)

model = test_GPU_CNN_model

In [8]:
from model.chrome_vision import ChromeVisionModel

model = ChromeVisionModel(
    input_shape= 1024 * 2048,    
    hidden_units=5,    # Units in the hidden layer
    output_shape= 3 # one for every class (Arbitrary right now)
).to(device)

#### MoCo

In [9]:
from model.chrome_vision import ChromeCut
from model.encoder import ResNet50

encoder = ResNet50(num_classes=len(MNIST_class_names),
                   in_features=1000)

model = ChromeCut(base_encoder=encoder,
                  feature_dim=len(MNIST_class_names),
                  queue_size=5000,
                  momentum=0.9,
                  softmax_temp=0.07,
                  mlp=True).to(device)

### Setup loss function and optimizer

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)

### Training loop - MoCo

In [11]:
from timeit import default_timer as timer
from tqdm.auto import tqdm
from model.evaluation import train_step, test_step # use torchmetrics.utilities.data.select_topk
from model.utilis import print_train_time, accuracy_top_k

torch.manual_seed(42)
train_time_start_on_cpu = timer()

epochs = 3
for epoch in tqdm(range(epochs)):
    print(f'Epoch: {epoch}\n')

    train_step(model=model,
               data_loader=MNIST_train_dataloader,
               loss_fn=loss_fn,
               optimizer=optimizer,
               accuracy_fn=accuracy_top_k,
               device=device)
    
    test_step(model=model,
               data_loader=MNIST_test_dataloader,
               loss_fn=loss_fn,
               accuracy_fn=accuracy_top_k,
               device=device)

# Print time taken
train_time_end_on_cpu = timer()
total_train_time_model = print_train_time(train_time_start_on_cpu, train_time_end_on_cpu, str(next(model.parameters()).device))

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0



  0%|          | 0/12000 [00:00<?, ?it/s]

### Training loop - For testing CPU and GPU (do not change)

In [21]:
from timeit import default_timer as timer
from tqdm.auto import tqdm
from model.evaluation import train_step_label, test_step_label # use torchmetrics.Accuracy()
from model.utilis import print_train_time, accuracy_fn

torch.manual_seed(42)
train_time_start_on_cpu = timer()

epochs = 3
for epoch in tqdm(range(epochs)):
    print(f'Epoch: {epoch}\n')

    train_step_label(model=model,
               data_loader=MNIST_train_dataloader,
               loss_fn=loss_fn,
               optimizer=optimizer,
               accuracy_fn=accuracy_fn,
               device=device)
    
    test_step_label(model=model,
               data_loader=MNIST_test_dataloader,
               loss_fn=loss_fn,
               accuracy_fn=accuracy_fn,
               device=device)

# Print time taken
train_time_end_on_cpu = timer()
total_train_time_model = print_train_time(train_time_start_on_cpu, train_time_end_on_cpu, str(next(model.parameters()).device))

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0

Train loss: 1.15849 | Train acc: 58.56%


  0%|          | 0/2000 [00:00<?, ?it/s]

Epoch: 1

Train loss: 1.03261 | Train acc: 61.05%


  0%|          | 0/2000 [00:00<?, ?it/s]

Epoch: 2

Train loss: 1.00766 | Train acc: 61.72%


  0%|          | 0/2000 [00:00<?, ?it/s]

Train time on cuda:0: 49.744 seconds


### Calculate model results on test dataset

In [ ]:
model_results = test_step(model=model,
                           data_loader=MNIST_test_dataloader,
                           loss_fn=loss_fn,
                           accuracy_fn=accuracy_top_k,
                           device=device)

model_results